### Importing the Libraries

In [1]:
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd

### Reading the PFD file

In [2]:
reader = PdfReader("Responses.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()
texts = np.array(text.split('?'))
print(texts)

FileNotFoundError: [Errno 2] No such file or directory: 'UResponses.pdf'

In [ ]:
type(texts)

In [ ]:
print(texts.shape)

### Cleaning the Data

In [ ]:
word = "Timestamp "
texts[0] = texts[0].replace(word,"")
print(texts)

In [ ]:
index = 15
ques = texts[:index]
feedback = texts[index:]
print("Questions =",ques)
print("Feedbacks =",feedback)

In [ ]:
ques.shape

In [ ]:
df_col = ques
df_col

In [ ]:
df_col.shape

In [ ]:
s_fb = np.char.split(feedback)
feedback = np.concatenate(s_fb)
print(feedback)

In [ ]:
feedback.shape

In [ ]:
import re
stu_fbs = feedback.copy()
DP = r"\b\d{2}/\d{2}/\d{4}\b"
TP = r"\b\d{2}:\d{2}:\d{2}\b"
for i in range(len(stu_fbs)):
    stu_fbs[i] = re.sub(DP,"",stu_fbs[i])
    stu_fbs[i] = re.sub(TP,"",stu_fbs[i])

print(stu_fbs)

In [ ]:
word = "5/13/2023"
stu_fbs[0] = stu_fbs[0].replace(word,"")
print(stu_fbs)

In [ ]:
index = 15
feedback_ans = texts[index:]
feedback_ans

In [ ]:
feedback_ans
words_to_keep = ["Arithmetic","All of the above","Algebra","Trigonometry","Geometry","Moderately Agree","Strongly Agree","Not yet decided","Strongly disagree","Strongly Disagree","Moderately Disagree","Not at all","Some times","No, I didn't","Yes, I suffered","None of the above","Sometimes","Yes","never"]
#words_to_keep = [word.lower() for word in words_to_keep]
join_arr = ' '.join(feedback_ans)
for phrase in words_to_keep:
    if ' ' in phrase:
        join_arr = join_arr.replace(phrase,phrase.replace(' ','_'))

spl_arr = join_arr.split()

fil_arr = [word for word in spl_arr if word in words_to_keep or word.replace('_', ' ') in words_to_keep]

fil_arr = [word.replace('_', ' ') for word in fil_arr]

print(fil_arr)

In [ ]:
fil_arr
specific_word = ["Arithmetic","All of the above","Algebra","Trigonometry","Geometry","Moderately Agree","Strongly Agree","Not yet decided","Strongly disagree","Strongly Disagree","Moderately Disagree","Not at all","Some times","No, I didn't","Yes, I suffered","None of the above","Sometimes","Yes","never"]
nested_arr = [[]]
for word in fil_arr:
    if word in specific_word:
        nested_arr.append([word])
    else:
        nested_arr[-1].append(word)

#nested_arr = np.array(nested_arr)
print(nested_arr)

In [ ]:
nested_arr = [lst for lst in nested_arr if lst]
print(nested_arr)

In [ ]:
# Words to start a new list
start_words = ["Arithmetic", "Algebra", "Geometry", "Trigonometry","All of the above"]

# Initialize an empty list to hold your nested lists
nested_list = []
# Initialize an empty list to hold the current sublist
sublist = []

# Iterate over each list in the current list
for lst in nested_arr:
    # If the first element of the list is a start word, start a new sublist
    if lst[0] in start_words:
        # If the current sublist is not empty, add it to the nested list
        if sublist:
            nested_list.append(sublist)
        # Start a new sublist with the current list
        sublist = lst
    else:
        # If the first element of the list is not a start word, add the list to the current sublist
        sublist += lst

# Add the last sublist to the nested list
if sublist:
    nested_list.append(sublist)

print(nested_list)


In [ ]:
type(nested_list)

In [ ]:
# Assuming 'nested_list' is your list
num_rows = len(nested_list)
num_cols = len(nested_list[0]) if nested_list else 0  # Check if the list is not empty

print("Number of rows (sublists):", num_rows)
print("Number of columns (elements in a sublist):", num_cols)


### Converting into Data Frame

In [ ]:
df_row = pd.DataFrame(nested_list)

In [ ]:
df_row

In [ ]:
df_col

In [ ]:
col_df = pd.DataFrame(df_col.reshape(1,-1))
print(col_df)

In [ ]:
col_df.head()

In [ ]:
merge_df = pd.concat([col_df,df_row],ignore_index=True)
merge_df.head()

In [ ]:
merge_df

In [ ]:
merge_df.columns = merge_df.iloc[0]
merge_df = merge_df[1:]
merge_df = merge_df.reset_index(drop=True)
final_df = merge_df

In [ ]:
final_df

### Applying Sentiment Analysis

In [ ]:
final_df['Which part of Math is terrifying for you'].value_counts().sort_index()

In [ ]:
from textblob import TextBlob

opt1 = TextBlob("Algebra")
opt2 = TextBlob("All of the above")
opt3 = TextBlob("Arithmetic")
opt4 = TextBlob("Geometry")
opt5 = TextBlob("Trigonometry")

print(opt1.sentiment)
print(opt2.sentiment)
print(opt3.sentiment)
print(opt4.sentiment)
print(opt5.sentiment)

In [ ]:
plot1 = final_df['Which part of Math is terrifying for you'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the first question')

In [ ]:
final_df[' Do you feel that unavailability of good Math teacher is a reason behind your fear of math'].value_counts().sort_index()

In [ ]:
from textblob import TextBlob

opt1 = TextBlob("Moderately Agree")
opt2 = TextBlob("Moderately Disagree")
opt3 = TextBlob("Not yet decided")
opt4 = TextBlob("Strongly Agree")
opt5 = TextBlob("Strongly disagree")

print(opt1.sentiment)
print(opt2.sentiment)
print(opt3.sentiment)
print(opt4.sentiment)
print(opt5.sentiment)

In [ ]:
sen_ana_feed2 = final_df[' Do you feel that unavailability of good Math teacher is a reason behind your fear of math']
feedback_count = final_df[' Do you feel that unavailability of good Math teacher is a reason behind your fear of math'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed2.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you feel that unavailability of good Math teacher is a reason behind your fear of math")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot2 = final_df[' Do you feel that unavailability of good Math teacher is a reason behind your fear of math'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the second question')

In [ ]:
final_df['  Do you get frustrated or frightened when you fail to solve problems repeatedly'].value_counts().sort_index()

In [ ]:
sen_ana_feed3 = final_df['  Do you get frustrated or frightened when you fail to solve problems repeatedly']
feedback_count = final_df['  Do you get frustrated or frightened when you fail to solve problems repeatedly'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed3.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print("Do you get frustrated or frightened when you fail to solve problems repeatedly")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot3 = final_df['  Do you get frustrated or frightened when you fail to solve problems repeatedly'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the third question')

In [ ]:
final_df[' Do you feel rote learning of mathematics increase Math phobia'].value_counts().sort_index()

In [ ]:
opt1 = TextBlob("No, I didn't")
opt2 = TextBlob("Not at all")
opt3 = TextBlob("Some times")
print(opt1.sentiment)
print(opt2.sentiment)
print(opt3.sentiment)

In [ ]:
sen_ana_feed4 = final_df[' Do you feel rote learning of mathematics increase Math phobia']
feedback_count = final_df[' Do you feel rote learning of mathematics increase Math phobia'].value_counts()
sentiment_scores = {
    "Strongly Agree": 3,
    "Moderately Agree": 2,
    "Not yet decided": 0,
    "Moderately Disagree": -2,
    "Strongly Disagree": -3,
    "No, I didn't": 0.5,
    "Not at all": -1,
    "Some times": 1
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed4.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you feel rote learning of mathematics increase Math phobia")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot4 = final_df[' Do you feel rote learning of mathematics increase Math phobia'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the fourth question')

In [ ]:
final_df[' Do you have Dyscalculia'].value_counts().sort_index()

In [ ]:
sen_ana_feed5 = final_df[' Do you have Dyscalculia']
feedback_count = final_df[' Do you have Dyscalculia'].value_counts()
sentiment_scores = {
    "Strongly Agree": 3,
    "Moderately Agree": 2,
    "Not yet decided": 0,
    "Moderately Disagree": -2,
    "Strongly disagree": -3,
    "No, I didn't": 0.5,
    "None of the above": 0,
    "Not at all": -1,
    "Some times": 1,
    "Yes, I suffered": -0.5
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed5.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you have Dyscalculia")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot5 = final_df[' Do you have Dyscalculia'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the fifth question')

In [ ]:
final_df[' Do you think psychological disorder is one of the reasons behind Math phobia'].value_counts().sort_index()

In [ ]:
sen_ana_feed6 = final_df[' Do you think psychological disorder is one of the reasons behind Math phobia']
feedback_count = final_df[' Do you think psychological disorder is one of the reasons behind Math phobia'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed6.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you think psychological disorder is one of the reason behind Math phobia")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot6 = final_df[' Do you think psychological disorder is one of the reasons behind Math phobia'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the sixth question')

In [ ]:
final_df[' Do you think lack of concentration is one of the reasons behind your fear of math'].value_counts().sort_index()

In [ ]:
sen_ana_feed7 = final_df[' Do you think lack of concentration is one of the reasons behind your fear of math']
feedback_count = final_df[' Do you think lack of concentration is one of the reasons behind your fear of math'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed7.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you think lack of conventration is one of the reasons behind your fear of math")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot7 = final_df[' Do you think lack of concentration is one of the reasons behind your fear of math'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the seventh question')

In [ ]:
final_df[' Do you think unclear concepts of the topics are the reasons behind Math phobia'].value_counts().sort_index()

In [ ]:
sen_ana_feed8 = final_df[' Do you think unclear concepts of the topics are the reasons behind Math phobia']
feedback_count = final_df[' Do you think unclear concepts of the topics are the reasons behind Math phobia'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed8.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you think unclear concepts of the topics are the reasons behind Math phobia")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot8 = final_df[' Do you think unclear concepts of the topics are the reasons behind Math phobia'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the eighth question')

In [ ]:
final_df[' Do you think Yoga can solve the Math phobia'].value_counts().sort_index()

In [ ]:
sen_ana_feed9 = final_df[' Do you think Yoga can solve the Math phobia']
feedback_count = final_df[' Do you think Yoga can solve the Math phobia'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed9.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you think Yoga can solve the Math phobia")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot9 = final_df[' Do you think Yoga can solve the Math phobia'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the ninth question')

In [ ]:
final_df[' Do you think Math phobia is caused due to the pressure of performing well'].value_counts().sort_index()

In [ ]:
sen_ana_feed10 = final_df[' Do you think Math phobia is caused due to the pressure of performing well']
feedback_count = final_df[' Do you think Math phobia is caused due to the pressure of performing well'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed10.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you think Math phobia is caused due to the presure of performing well")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot10 = final_df[' Do you think Math phobia is caused due to the pressure of performing well'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the tenth question')

In [ ]:
final_df[' Do you feel Mathematics phobia arises because you are not able to connect Mathematics with the real world'].value_counts().sort_index()

In [ ]:
sen_ana_feed11 = final_df[' Do you feel Mathematics phobia arises because you are not able to connect Mathematics with the real world']
feedback_count = final_df[' Do you feel Mathematics phobia arises because you are not able to connect Mathematics with the real world'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed11.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you feel that unavailability of good Math teacher is a reason behind your fear of math")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot11 = final_df[' Do you feel Mathematics phobia arises because you are not able to connect Mathematics with the real world'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the eleventh question')

In [ ]:
final_df[' Do you agree that regular practice of Mathematics can solve the problems related to Math phobia'].value_counts().sort_index()

In [ ]:
sen_ana_feed12 = final_df[' Do you agree that regular practice of Mathematics can solve the problems related to Math phobia']
feedback_count = final_df[' Do you agree that regular practice of Mathematics can solve the problems related to Math phobia'].value_counts()
sentiment_scores = {
    "Strongly Agree": 2,
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed12.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Do you agree that regular practice of Mathematics can solve the problems related to Math phobia")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot12 = final_df[' Do you agree that regular practice of Mathematics can solve the problems related to Math phobia'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the twelveth question')

In [ ]:
final_df[' Does breaking up a complicated problem into smaller part help in betterment'].value_counts().sort_index()

In [ ]:
sen_ana_feed13 = final_df[' Does breaking up a complicated problem into smaller part help in betterment']
feedback_count = final_df[' Does breaking up a complicated problem into smaller part help in betterment'].value_counts()
sentiment_scores = {
    "Strongly Agree": 3,
    "Moderately Agree": 2,
    "Not yet decided": 0,
    "Moderately Disagree": -2,
    "Strongly Disagree": -3,
    "Sometimes": 0,
    "Yes": 1,
    "never": -1 
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed13.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Does breaking up a complicated problem into smaller part help in betterment")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot13 = final_df[' Does breaking up a complicated problem into smaller part help in betterment'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the thirteenth question')

In [ ]:
final_df[' Does studying in peer groups and seeking help from teacher help to eradicate Math phobia'].value_counts().sort_index()

In [ ]:
sen_ana_feed14 = final_df[' Does studying in peer groups and seeking help from teacher help to eradicate Math phobia']
feedback_count = final_df[' Does studying in peer groups and seeking help from teacher help to eradicate Math phobia'].value_counts()
sentiment_scores = {
    "Moderately Agree": 3,
    "Moderately Disagree": -1,
    "Sometimes": 2,
    "Strongly Disagree": -2,
    "Yes": 1,
    "never": -3
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed14.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']/sum(feedback_count.values)

print(" Does studying in peer groups and seeking help from teacher help to eradicate Math phobia")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot14 = final_df[' Does studying in peer groups and seeking help from teacher help to eradicate Math phobia'].value_counts().sort_index().plot(kind='pie', title='Number of responses for forteenth question')

In [ ]:
final_df[' Does frequent mock tests help to release the pressure'].value_counts().sort_index()

In [ ]:
sen_ana_feed15 = final_df[' Does frequent mock tests help to release the pressure']
feedback_count = final_df[' Does frequent mock tests help to release the pressure'].value_counts()
sentiment_scores = {
    "Moderately Agree": 1,
    "Not yet decided": 0,
    "Moderately Disagree": -1,
    "Strongly Disagree": -2
}

sentiment_score_df = pd.DataFrame({
    'feedback': feedback_count.index,
    'count': feedback_count.values 
})

final_df['sentiment_score'] = sen_ana_feed15.map(sentiment_scores)
sentiment_score_df['sentiment_score'] = sentiment_score_df['feedback'].map(sentiment_scores)/sum(feedback_count.values)

# Calculate the average sentiment score
average_sentiment_score = final_df['sentiment_score'].mean()
sentiment_score_df['total_sentiment_score'] = sentiment_score_df['count'] * sentiment_score_df['sentiment_score']

print(" Does frequent mock tests help to release the pressure")
print('----------------------------------------------------------------------------------------------------------------------')
print(sentiment_score_df)
print('----------------------------------------------------------------------------------------------------------------------')
print("The average sentiment score is:", average_sentiment_score)

In [ ]:
plot15 = final_df[' Does frequent mock tests help to release the pressure'].value_counts().sort_index().plot(kind='pie', title='Number of responses for the fifteenth question')

In [ ]:
from textblob import TextBlob

def analyze_sentiment(final_df, row_index):
    sentiments = []
    for question in final_df.columns:
        feedback = str(final_df.loc[row_index, question])
        sentiment = TextBlob(feedback).sentiment.polarity
        sentiments.append(sentiment)
    return sentiments


In [ ]:
row_index = 17 # change this to the index of the row you want to analyze
sentiments = analyze_sentiment(final_df, row_index)
res = sum(sentiments)/15
if 0<= res <=0.12:
    print("Confident")
else:
    print("Under-Confident")

print("Sttudent sentiment analysis score:",res)